# Data Understanding

## Preperation

Import packages and set globals

In [1]:
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option("display.width", 1000)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format

%matplotlib inline
plt.rcParams['figure.figsize'] = (20, 6)

Importing the gdp.pkl file

In [2]:
df = pd.read_pickle("../data/raw/gdp.pkl")

## Structure

Brief overview of the dataset

### Data Format

In [3]:
df.head(10)

,FIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,2017,2018,2019,2020,2021,2022
0,0,United States,,CAGDP9,1,...,All industry total,Thousands of chained 2017 dollars,19612102000.00,20193896000.00,20692087000.00,20234074000.00,21407692000.00,21822037000.00
1,0,United States,,CAGDP9,2,...,Private industries,Thousands of chained 2017 dollars,17156255000.00,17711775000.00,18195752000.00,17761871000.00,18909453000.00,19283116000.00
2,0,United States,,CAGDP9,3,11,"Agriculture, forestry, fishing and hunting",Thousands of chained 2017 dollars,176840000.00,184105000.00,171149000.00,173659000.00,183742000.00,170058000.00
3,0,United States,,CAGDP9,6,21,"Mining, quarrying, and oil and gas extraction",Thousands of chained 2017 dollars,267302000.00,277013000.00,314604000.00,305588000.00,269478000.00,245290000.00
4,0,United States,,CAGDP9,10,22,Utilities,Thousands of chained 2017 dollars,313711000.00,309269000.00,312920000.00,332336000.00,316603000.00,318392000.00
5,0,United States,,CAGDP9,11,23,Construction,Thousands of chained 2017 dollars,840220000.00,863755000.00,882046000.00,856487000.00,888104000.00,827768000.00
6,0,United States,,CAGDP9,12,31-33,Manufacturing,Thousands of chained 2017 dollars,2109718000.00,2213031000.00,2223326000.00,2127060000.00,2248052000.00,2277757000.00
7,0,United States,,CAGDP9,13,"321,327-339",Durable goods manufacturing,Thousands of chained 2017 dollars,1178271000.00,1227629000.00,1241367000.00,1171720000.00,1249509000.00,1296970000.00
8,0,United States,,CAGDP9,25,"311-316,322-326",Nondurable goods manufacturing,Thousands of chained 2017 dollars,931447000.00,985229000.00,981959000.00,955683000.00,999648000.00,985406000.00
9,0,United States,,CAGDP9,34,42,Wholesale trade,Thousands of chained 2017 dollars,1176146000.00,1185896000.00,1191436000.00,1200151000.00,1192824000.00,1146825000.00


In [4]:
df.dtypes

FIPS                        int64
GeoName                    object
Region                     object
TableName                  object
LineCode                    int64
IndustryClassification     object
Description                object
Unit                       object
2017                      float64
2018                      float64
2019                      float64
2020                      float64
2021                      float64
2022                      float64
dtype: object

### Remove unnecessary data for our task

"Over-aggregated" records via FIPS:
- US-wide GDPs
- FIPS totals
- Private totals

NAICS which don't fit our market:
- not in metal working
- focus on 11, 21, 22, 23, 31-33

In [5]:
# Remove FIPS as stated
df = df[df["FIPS"] != 0]
df = df[df["IndustryClassification"] != "..."]

# Remove NAICS as stated
naics_filter = "|".join(["^11", "^21", "^22", "^23", "^31", "^32", "^33"])
df = df.loc[df["IndustryClassification"].str.contains(naics_filter)]

### Content Description

In [6]:
df.describe()

,FIPS,LineCode,2017,2018,2019,2020,2021,2022
count,31770.00,31770.00,26553.00,26489.00,26249.00,26462.00,26570.00,27115.00
mean,30301.32,34.60,1150981.64,1201118.54,1237101.28,1194253.41,1253288.36,1240593.52
std,15461.83,35.81,12656432.93,13303683.52,13785083.66,13426488.02,14467432.22,14575318.89
min,1000.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,18157.00,10.00,9377.00,9572.00,9810.00,10062.25,11193.75,10048.00
50%,29151.00,12.50,45309.00,46404.00,47052.00,47401.50,52661.00,49690.00
75%,45043.00,87.00,207074.00,214071.00,216414.00,208956.50,215579.25,213971.00
max,98000.00,90.00,746914680.00,797666904.00,859510532.00,877152423.00,986444700.00,1010180851.00


In [7]:
df.isnull().sum()

FIPS                         0
GeoName                      0
Region                       0
TableName                    0
LineCode                     0
IndustryClassification       0
Description                  0
Unit                         0
2017                      5217
2018                      5281
2019                      5521
2020                      5308
2021                      5200
2022                      4655
dtype: int64

## Data Analysis

Top 10 industries by generated value in ***2022***

In [8]:
highest_gdp = df.groupby(["IndustryClassification", "Description"])[["2017", "2018", "2019", "2020", "2021", "2022"]].sum().reset_index()
highest_gdp = highest_gdp.sort_values(by="2022", ascending=False).reset_index()

highest_gdp.head(10)

,index,IndustryClassification,Description,2017,2018,2019,2020,2021,2022
0,7,"31-33,51",Manufacturing and information,9280243920.00,9735987237.00,10083971728.00,9947999865.00,10806349751.00,11233345677.00
1,6,31-33,Manufacturing,6303678118.00,6607936635.00,6642889526.00,6353740767.00,6726617219.00,6809586832.00
2,9,"321,327-339",Durable goods manufacturing,3528738656.00,3675680699.00,3718305615.00,3512050183.00,3746374584.00,3892390541.00
3,8,"311-316,322-326",Nondurable goods manufacturing,2786092512.00,2945577109.00,2933192468.00,2850567220.00,2989384313.00,2940975130.00
4,4,"22,48-49",Transportation and utilities,2663723921.00,2691656290.00,2721750645.00,2620685496.00,2813822670.00,2933333511.00
5,5,23,Construction,2504011752.00,2575377369.00,2632920205.00,2556998503.00,2651922125.00,2471916270.00
6,1,"11,21",Natural resources and mining,1275530786.00,1329960208.00,1399298392.00,1384137916.00,1312505437.00,1200288749.00
7,3,22,Utilities,913465796.00,898899015.00,909249001.00,962765669.00,921435679.00,939502674.00
8,2,21,"Mining, quarrying, and oil and gas extraction",801886963.00,830967240.00,943847461.00,917071685.00,780216143.00,707038921.00
9,0,11,"Agriculture, forestry, fishing and hunting",504642996.00,524387312.00,487246501.00,496316353.00,551243827.00,510315065.00


Top 10 industries by ***mean gdp over 6 years***

In [9]:
highest_gdp['mean_gdp'] = highest_gdp[["2017", "2018", "2019", "2020", "2021", "2022"]].mean(axis=1)
highest_gdp = highest_gdp.sort_values(by="mean_gdp", ascending=False).reset_index()

highest_gdp.head(10)

,level_0,index,IndustryClassification,Description,2017,2018,2019,2020,2021,2022,mean_gdp
0,0,7,"31-33,51",Manufacturing and information,9280243920.00,9735987237.00,10083971728.00,9947999865.00,10806349751.00,11233345677.00,10181316363.00
1,1,6,31-33,Manufacturing,6303678118.00,6607936635.00,6642889526.00,6353740767.00,6726617219.00,6809586832.00,6574074849.50
2,2,9,"321,327-339",Durable goods manufacturing,3528738656.00,3675680699.00,3718305615.00,3512050183.00,3746374584.00,3892390541.00,3678923379.67
3,3,8,"311-316,322-326",Nondurable goods manufacturing,2786092512.00,2945577109.00,2933192468.00,2850567220.00,2989384313.00,2940975130.00,2907631458.67
4,4,4,"22,48-49",Transportation and utilities,2663723921.00,2691656290.00,2721750645.00,2620685496.00,2813822670.00,2933333511.00,2740828755.50
5,5,5,23,Construction,2504011752.00,2575377369.00,2632920205.00,2556998503.00,2651922125.00,2471916270.00,2565524370.67
6,6,1,"11,21",Natural resources and mining,1275530786.00,1329960208.00,1399298392.00,1384137916.00,1312505437.00,1200288749.00,1316953581.33
7,7,3,22,Utilities,913465796.00,898899015.00,909249001.00,962765669.00,921435679.00,939502674.00,924219639.00
8,8,2,21,"Mining, quarrying, and oil and gas extraction",801886963.00,830967240.00,943847461.00,917071685.00,780216143.00,707038921.00,830171402.17
9,9,0,11,"Agriculture, forestry, fishing and hunting",504642996.00,524387312.00,487246501.00,496316353.00,551243827.00,510315065.00,512358675.67


## Export

Import top_industries.pkl

In [10]:
top_industries = pd.read_pickle("../data/processed/top_industries.pkl")

Export the top 5 industries into top_industries.pkl

In [11]:
top_industries["by_gdp"] = highest_gdp["IndustryClassification"].head(5)

In [12]:
pd.to_pickle(top_industries, "../data/processed/top_industries.pkl")